<a href="https://colab.research.google.com/github/ornenovino/python_course_eim/blob/main/intro_py_m4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la programación en python

## Módulo 4: Análisis de Texto y Webscrapping

Introducción al análisis de texto y cómo utilizar Python para realizar webscrapping.

Para este módulo utilizaremos dos librerías nuevas `requests` para realizar peticiones del tipo `GET` a un servidor donde va a estar alojada una página web y `BeautifulSoup` que nos permite acceder a los distintos `<tags>` o elementos `HTML`.

In [ ]:
import requests
from bs4 import BeautifulSoup

### Scrapping simple

In [ ]:
# Para comenzar de manera simple tomamos un URL que contiene una tabla 
# de peliculas alojada en IMBD

url = "https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv" # colocamos el URL en una variable
page = requests.get(url) # hacemos un request de tipo GET al servidor
soup = BeautifulSoup(page.content, 'html.parser') # salvamos el html.parser que nos permitira acceder a los elementos del maquetado web

# con un simple comando podemos acceder y guardar los títulos de las peliculas
# sabiendo que los mismos son h3
movies = soup.find_all('h3', class_='lister-item-header')
movies

In [ ]:
# Creamos una lista vacia
movie_data = []

for movie in movies:  # hacemos una estructura de control que itere sobre la lista anterior movies

    # nos quedamos con el titulo y lo guardamos
    title = movie.find('a').text

    # nos quedamos con el año y lo guardamos
    year = movie.find('span', class_='lister-item-year text-muted unbold').text

    # hacemos un append de ambos valores a nuestra lista inicial
    movie_data.append((title, year))

# hacemos un print de lo que se salvo en la lista
print(movie_data)

### Scrapping complejo y dataframes

In [ ]:
# importamos pandas
import pandas as pd

In [ ]:
# sobreescribimos variables de interes
page = requests.get("https://www.imdb.com/chart/top")
soup = BeautifulSoup(page.content, 'html.parser')

# creamos la lista vacia
movies_list = []

In [ ]:
# encontramos la tabla en la pagina web
tbody = soup.find('tbody', class_='lister-list')
trs = tbody.find_all('tr')  # encontramos todas las filas

for tr in trs:  # iteramos por sobre todas las filas

    # nos quedamos con el titulo y el año de cada pelicula
    title = tr.find('td', class_='titleColumn').a.text
    year = tr.find('td', class_='titleColumn').span.text[1:-1]

    # guardamos el rating
    rating = float(tr.find('td', class_='ratingColumn imdbRating').strong.text)

    # hacemos append de cada una de estas variables
    movies_list.append((title, year, rating))

In [ ]:
# creamos un pandas dataframe de la lista anterior definiendo las columnas
movies_df = pd.DataFrame(movies_list, columns=['Title', 'Year', 'Rating'])

# incluso lo podemos salvar como un CSV
movies_df.to_csv('imdb-movies-dataset.csv', index = False)

In [ ]:
# podemos echar un vistazo del df que acabamos de crear
movies_df.head()

### Análisis de texto

**Nube de palabras**

In [ ]:
# importamos las librerias
import nltk 
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt

ed_sheeran =  pd.read_csv('/content/drive/MyDrive/python_intro/EdSheeran.csv', encoding="utf-8")
ed_sheeran.head()

In [ ]:
# vamos a tomar la letra de la cancion Shape of You
pd.set_option('display.max_colwidth', None)

shape_of_you = ed_sheeran[ed_sheeran['Title'] == 'Shape of You']
lyrics = shape_of_you['Lyric'].to_string()
type(lyrics)

In [ ]:
# tokenizing la letra por palabra
tokens = word_tokenize(lyrics)

# creamos una distrbucion simple de las palabras
freq_dist = nltk.FreqDist(tokens) 
  
# mostramos las 5 palabras más frecuentes en la letra
print(freq_dist.most_common(5))

Como notamos la letra contiene un montón de conectores que se deberían de eliminar.

In [ ]:
from nltk.corpus import stopwords
import string
nltk.download('stopwords')

# removemos puntuacion
remove_punctuation = [char for char in lyrics if char not in string.punctuation] 

# agregamos de nuevo a la lista sin puntuacion
remove_punctuation = ''.join(remove_punctuation) 

# removemos las conocidas 'stopwords'
stop_words = set(stopwords.words('english')) 

# hacemos la tokenizacion nuevamente
word_tokens = word_tokenize(remove_punctuation) 

# realizamos un condicional como list comprehension
filtered_sentence = [w for w in word_tokens if not w in stop_words]

In [ ]:
print(len(tokens))
print(len(filtered_sentence)) # redujimos hasta la mitad las palabras
filtered_sentence

In [ ]:
freq_dist_clean = nltk.FreqDist(filtered_sentence)
print(freq_dist_clean.most_common(5)) # cambian completamente las palabras mas frecuentes

In [ ]:
# podemos generar una nube de palabras
wordcloud = WordCloud(width = 800, 
                      height = 800,
                      background_color ='white', 
                      min_font_size = 10).generate(" ".join(filtered_sentence)) 

# realizamos un plot de la misma
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show()

**Sentimientos**

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
filtered_to_string = " ".join(filtered_sentence)
filtered_to_string

In [ ]:
# creamos una función para analizar los sentimientos

def sentiment_scores(sentence):
 
    # creamos un objeto del tipo SentimentIntensityAnalyze
    sid_obj = SentimentIntensityAnalyzer()
 
    # usamos el método polarity_scores del SentimentIntensityAnalyzer
    # que nos da un diccionario de sentimientos
    # que contiene pos (positivos) neg (negativos) neu(neutros) y los el score
    # el compound es la suma de los positivos, negativos y neutrales
    # normalizado entre -1 extremo negativo y +1 extremo positivo
    
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
    print("Los scores son: ", sentiment_dict)
    print("La letra fue un ", sentiment_dict['neg']*100, "% Negativa")
    print("La letra fue un  ", sentiment_dict['neu']*100, "% Neutral")
    print("La letra fue un  ", sentiment_dict['pos']*100, "% Positiva")
 
    print("Entonces la canción es", end = " ")
 
    # imprimimos el resultado de acuerdo a donde caiga el número
    if sentiment_dict['compound'] >= 0.05 :
        print("Positiva")
 
    elif sentiment_dict['compound'] <= - 0.05 :
        print("Negativa")
 
    else :
        print("Neutral")

In [ ]:
sentiment_scores(filtered_to_string)